In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'c:\\Users\\5300\\Desktop\\image_classification'

In [4]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Arshavin023/deep-learning---image-classification-with-CNN.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Arshavin023"
os.environ["MLFLOW_TRACKING_PASSWORD"]="3a6add494bc8a69b0623dd2f8030d353dde15622"

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [7]:
# Add entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model = "artifacts/training/model.h5",
            training_data = "artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri = "https://dagshub.com/Arshavin023/deep-learning---image-classification-with-CNN.mlflow",
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE)

        return eval_config

In [10]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [11]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config=config
    
    def valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
    

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()
    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_artifact_uri()).scheme

        if mlflow.active_run():
    # If there is an active run, end it
            mlflow.end_run()

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work file store
            if tracking_url_type_store != "file":
                # Register the model
                mlflow.keras.log_model(self.model, "model", registered_model_name='VGG16Model')
            else:
                mlflow.keras.log_model(self.model, "model")


In [12]:
# Evaluation Pipeline
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2023-12-30 16:50:39,989: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-30 16:50:40,006: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-30 16:50:40,008: INFO: common: created directory at: artifacts]


Found 139 images belonging to 2 classes.
9/9 [==============================] - 23s 3s/step - loss: 9.4984 - accuracy: 0.5468
[2023-12-30 16:51:03,847: INFO: common: json file saved at: scores.json]


2023/12/30 16:51:07 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2023-12-30 16:51:11,458: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\5300\AppData\Local\Temp\tmpart98109\model\data\model\assets
[2023-12-30 16:51:13,445: INFO: builder_impl: Assets written to: C:\Users\5300\AppData\Local\Temp\tmpart98109\model\data\model\assets]


c:\Users\5300\AppData\Local\anaconda3\envs\kidney\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2023/12/30 16:52:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 3
Created version '3' of model 'VGG16Model'.
